# Types, Monads, and Machine Learning in Python

A declarative programming approach involves defining structures and describing transformations on those structures in an equational way. Using a declarative approach to implement a solution can have a number of benefits:

* the implementation is likely to be concise;
* it may be easier to quickly explore trade-offs between optimality and performance; and
* it may be easier to to store and later restart partial solutions.

In this article we will present a number of declarative Python solutions to a well-known optimization problem and illustrate how some of the above benefits may be realized. 

## Dependencies

In [259]:
# Presentation dependencies.
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib as mp
import matplotlib.pyplot as plt
from importlib import reload
from IPython.display import Image
from IPython.display import display_html
from IPython.display import display
from IPython.display import Math
from IPython.display import Latex
from IPython.display import HTML

# Content dependencies (also reproduced inline).
from numpy import np
from random import randint
from itertools import permutations
from functools import reduce
from tqdm import tqdm

ImportError: cannot import name 'np'

## Motivation: TensorFlow Example

In [260]:
import tensorflow as tf

# Initialize two constants.
x1 = tf.constant([1,2,3,4])
x2 = tf.constant([5,6,7,8])

print(x1)
print(x1 + x2) # returns: tf.Tensor([4 6], shape=(2,), dtype=int32)
print(x1 * x2)
print(2 * x2)


#mnist = tf.keras.datasets.mnist

#(x_train, y_train), (x_test, y_test) = mnist.load_data()
#x_train, x_test = x_train / 255.0, x_test / 255.0





# Intialize the Session.
#sess = tf.Session()

# Print the result.
#print(sess.run(result))

# Close the session.
#sess.close()

tf.Tensor([1 2 3 4], shape=(4,), dtype=int32)
tf.Tensor([ 6  8 10 12], shape=(4,), dtype=int32)
tf.Tensor([ 5 12 21 32], shape=(4,), dtype=int32)
tf.Tensor([10 12 14 16], shape=(4,), dtype=int32)


In [261]:
from __future__ import absolute_import, division, print_function, unicode_literals

mnist = tf.keras.datasets.mnist

((x_train, y_train), (x_test, y_test)) = mnist.load_data()
(x_train, x_test) = (x_train / 255.0, x_test / 255.0)

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
  )

model.fit(x_train, y_train, epochs=2)

model.evaluate(x_test, y_test)

Train on 60000 samples
Epoch 1/2
60000/60000 [==============================] - 7s 117us/sample - loss: 0.2931 - accuracy: 0.9155
Epoch 2/2
10000/10000 [==============================] - 1s 59us/sample - loss: 0.0967 - accuracy: 0.9707


[0.09666207031048835, 0.9707]

## Typing in Python

More information on the `typing` module can be found in the [documentation page](https://docs.python.org/3/library/typing.html). Note that there are no native, default, or "official" features for performing type checking on Python code.

In [243]:
# The typing module is included by default in Python 3.5 and higher.

def f(x: int) -> int:
    return x + x

print(f('abc')) # No actual static or dynamic checking.

abcabc
A type error occurred.


The [mypy library](http://www.mypy-lang.org/) can annotate Python code with types, type-check a program statically (without running it), or introduce dynamic (i.e., at runtime) type checking.

Other community members have written libraries to perform type checking (such as [enforce](https://github.com/RussBaz/enforce)), but none are "official" and some appear to be dormant as of 2019.

In [244]:
from enforce import runtime_validation

@runtime_validation
def g(x: int) -> int:
    return 2 * x

try:
    g('abc')
except:
    print("A type error occurred.")

A type error occurred.


We present some additional examples below.

In [245]:
from typing import Dict, Tuple, Sequence

def repeat(si: Tuple[str, int]) -> str:
    (s, i) = si
    return s * i

print(repeat(('xyz', 3)))

xyzxyzxyz


In [250]:
from typing import NewType

UserName = NewType('UserName', str)

def confirm(s: UserName) -> bool:
    return s == 'Alice'

print(confirm('Alice'))
print(confirm('Bob'))
print(confirm(123)) # No type checking by default.

True
False
False


## Monads

Monads are a way to organize modular software components that can be composed with one another like functions. The criteria for composing two components is that their types must match.

### State Monad for Logging

While it is not normally necessary to maintain state explicitly in a functional way in an imperative language like Python, it *can* be useful when writing multithreaded or distributed code. The state monad pattern is one way to deal with maintaining side effects (i.e., input streams, output streams, and so on).

In [219]:
from typing import Union, Callable, TypeVar

Log = NewType('Log', Sequence[str])

def divide_by_ten(n: int) -> Tuple[int, Log]:
    return (n // 10, ["Evaluated `divide_by_ten`."])

def add_two(n: int) -> int:
    return n + 2

# add_two(divide_by_ten(100)) # Error.



S = TypeVar('S')
T = TypeVar('T')

def lift(f: Callable[[S], T]) -> Callable[[Tuple[S, Log]], Tuple[T, Log]]:
    
    def lifted_f(x_log: Tuple[S, Log]) -> Tuple[T, Log]:
        (x, log) = x_log
        return (f(x), log)

    return lifted_f

def join(x_log_log: Tuple[Tuple[S, Log], Log]) -> Tuple[S, Log]:
    ((x, log1), log2) = x_log_log
    return (x, log1 + log2)

def intro(x: S) -> Tuple[S, Log]:
    return (x, [])

a: Tuple[int, Log] = intro(100)
b: Tuple[Tuple[int, Log], Log] = (lift(divide_by_ten))(a)
c: Tuple[int, Log] = join(b)
d: Tuple[int, Log] = (lift(add_two))(c)
d

abcabc
xyzxyzxyz
False
False


[125]

### Sequence/List Monad for Data Processing

The MapReduce paradigm actually corresponds to a sequence or list monad pattern.

In [254]:
# This is exactly the built-in map function.
def lift(f: Callable[[S], T]) -> Callable[[Sequence[S]], Sequence[T]]:
    
    def lifted_f(xs: Sequence[S]) -> Sequence[T]:
        return [f(x) for x in xs]

    return lifted_f

# This is the flatten function for nested lists.
# Note its relationship to the reduce function.
def join(xss: Sequence[Sequence[S]]) -> Sequence[S]:
    return [x for xs in xss for x in xs]

# Notice the relationship of this function to map.
def intro(x: S) -> Sequence[S]:
    return [x]

a: Sequence[int] = intro(123)
b: Sequence[int] = (lift(add_two))(a)
b

[125]

## Operator Overloading in Python

Python supports operator overloading, as well as overloading of common syntactic constructs (such as function application and indexing).

In [274]:
class Zoo():
    def __init__(self, *animals):
        self.animals = animals
    
    def __add__(self, other):
        return Zoo(*(self.animals + other.animals))

    def __lt__(self, other):
        return set(self.animals).issubset(set(other.animals))

    def __getitem__(self, animal):
        return animal in self.animals

    def __call__(self, animal):
        self.animals += tuple([animal])

    def __len__(self):
        return len(self.animals)
    
    def __str__(self):
        return str(self.animals)

    def __repr__(self):
        return str(self)
    
z1 = Zoo("Giraffe")
z2 = Zoo("Zebra", "Giraffe")
print(z1 + z2)
print(z1 < z2)
print(z1["Giraffe"])
print(z1["Zebra"])
z1("Lion")
print(z1)

('Giraffe', 'Zebra', 'Giraffe')
True
True
False
('Giraffe', 'Lion')


## Embedding a Language for Arithmetic Functions

We embed a simple language for integer data flows in Python.

In [230]:
class Var():
    def __init__(self, name):
        self.name = name

    def send(self, inputs):
        return inputs[self.name]

    def __repr__(self):
        return "Var('" + str(self.name) + "')"

class Sum():
    def __init__(self, *args):
        self.args = args

    def send(self, inputs):
        return sum([arg.send(inputs) for arg in self.args])
    
    def __repr__(self):
        return "Sum(" + (", ".join([str(arg) for arg in self.args])) + ")"

X = Var("X")
Y = Var("Y")

s = Sum(Sum(X, Y), Y)

print(s)
s.send({"X":22, "Y":33})

Sum(Sum(Var('X'), Var('Y')), Var('Y'))


88

We expand our example so that we can use arithmetic operator overloading to make our syntax more compact.

In [258]:
class Expr():
    def __add__(self, other):
        return Sum(self, other)
    
    def __mul__(self, other):
        return Prod(self, other)
    
    def __str__(self):
        return self.__repr__()

class Const(Expr):
    def __init__(self, constant):
        self.constant = constant
    
    def send(self, inputs):
        return self.constant
    
    def __repr__(self):
        return "Const(" + str(self.constant) + ")"

class Var(Expr):
    def __init__(self, name):
        self.name = name
    
    def send(self, inputs):
        return inputs[self.name]
    
    def __repr__(self):
        return "Var('" + str(self.name) + "')"

class Sum(Expr):
    def __init__(self, *args):
        self.args = args

    def send(self, inputs):
        return sum([arg.send(inputs) for arg in self.args])
    
    def __repr__(self):
        return "Sum(" + (", ".join([str(arg) for arg in self.args])) + ")"

class Fun(Expr):
    def __init__(self, name, function):
        self.name = name
        self.function = function
        self.args = None

    def __call__(self, *args):
        self.args = args
        return self

    def send(self, inputs):
        return self.function([arg.send(inputs) for arg in self.args])
    
    def __repr__(self):
        return self.name + "(" + (", ".join([str(arg) for arg in self.args])) + ")"

from functools import reduce
def prod(xs):
    return reduce(lambda x, y: x*y, xs)

X = Var("X")
Y = Var("Y")
Prod = Fun("Prod", prod)

s = Sum(Sum(X, Y), Y, Const(2))
p = Prod(X, Y, Const(2))

s.send({"X":22, "Y":33})
p.send({"X":2, "Y":3})
s

f = Sum(Prod(Var('K'), Var('X')), Var('C'))
f = (Var('K') * Var('X')) + Var('C')

We expand our example so that the data structure represents flows of *ranges* of integers (rather than individual ones).

In [257]:
from itertools import product

class Expr():
    def __add__(self, other):
        return Sum(self, other)
    
    def __mul__(self, other):
        return Prod(self, other)
    
    def __str__(self):
        return self.__repr__()

    def __call__(self, inputs):
        return self.send(inputs)

class Const(Expr):
    def __init__(self, constant):
        self.inputs = {}
        self.constant = constant
    
    def send(self, inputs):
        return [self.constant]
    
    def __call__(self, inputs):
        return self.send(inputs)
    
    def __repr__(self):
        return "Const(" + str(self.constant) + ")"

class Var(Expr):
    def __init__(self, name):
        self.inputs = {}
        self.name = name
    
    def send(self, inputs):
        return inputs[self.name]
    
    def __call__(self, inputs):
        return self.send(inputs)
    
    def __repr__(self):
        return "Var('" + str(self.name) + "')"

class Sum(Expr):
    def __init__(self, *args):
        self.inputs = {}
        self.args = args

    def send(self, inputs):
        list_of_arg_ranges = [arg.send(inputs) for arg in self.args]
        list_of_arg_combos = list(product(*list_of_arg_ranges))
        return [sum(c) for c in list_of_arg_combos]
    
    def __call__(self, inputs):
        return self.send(inputs)
    
    def __repr__(self):
        return "Sum(" + (", ".join([str(arg) for arg in self.args])) + ")"

class Fun(Expr):
    def __init__(self, name, function):
        self.inputs = {}
        self.name = name
        self.function = function
        self.args = None

    def send(self, inputs):
        list_of_arg_ranges = [arg.send(inputs) for arg in self.args]
        list_of_arg_combos = list(product(*list_of_arg_ranges))
        return [self.function(c) for c in list_of_arg_combos]

    def __call__(self, *args):
        if self.args is None:
            self.args = args
            return self
        else:
            return self.send(args)

    def __repr__(self):
        return self.name + "(" + (", ".join([str(arg) for arg in self.args])) + ")"

    
Flow = NewType('Flow', Callable[[Dict[str, int]], int])

# Const: int -> Callable[[Dict[str, int]], int]
C: Flow = Const(123)

# Var: str -> Callable[[Dict[str, int]], int]
V: Flow = Var("V")

# Sum: Sequence[Callable[[Dict[str, int]], int]] -> Callable[[Dict[str, int]], int]
# Sum: Sequence[Flow] -> Flow

# Fun: Callable[[int], int] -> (
#         Sequence[Callable[[Dict[str, int]], int]]
#         -> Callable[[Dict[str, int]], int]
#      )
#
# Fun: Callable[[int], int] -> (Sequence[Flow] -> Flow)

from functools import reduce
def prod(xs):
    return reduce(lambda x, y: x*y, xs)

Prod = Fun("Prod", prod)
f = (Var('K') * Var('X')) + Var('C')
f({'K':[1,2,3], 'X':[1,2,3], 'C':[1,2,3]})

[2,
 3,
 4,
 3,
 4,
 5,
 4,
 5,
 6,
 3,
 4,
 5,
 5,
 6,
 7,
 7,
 8,
 9,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12]

We can use our data structure to build a small optimization example.

In [256]:
data = [(x, x*x - 5) for x in range(1,6)]
(xs, ys) = zip(*data)

R = range(1,5)
models = [{'K':[k], 'C':[c]} for k in R for c in R]

m_ys_s = [(m, f({**m, **{'X':xs}})) for m in models]

def dist_sqr(v, w):
    return sum([(vi - wi)**2 for (vi, wi) in zip(v, w)])

m_err_s = [(m, dist_sqr(ys_, ys)) for (m,ys_) in m_ys_s]

m_err_s

[({'K': [1], 'C': [1]}, 284),
 ({'K': [1], 'C': [2]}, 269),
 ({'K': [1], 'C': [3]}, 264),
 ({'K': [1], 'C': [4]}, 269),
 ({'K': [2], 'C': [1]}, 179),
 ({'K': [2], 'C': [2]}, 194),
 ({'K': [2], 'C': [3]}, 219),
 ({'K': [2], 'C': [4]}, 254),
 ({'K': [3], 'C': [1]}, 184),
 ({'K': [3], 'C': [2]}, 229),
 ({'K': [3], 'C': [3]}, 284),
 ({'K': [3], 'C': [4]}, 349),
 ({'K': [4], 'C': [1]}, 299),
 ({'K': [4], 'C': [2]}, 374),
 ({'K': [4], 'C': [3]}, 459),
 ({'K': [4], 'C': [4]}, 554)]

This is the end of the document.